In [115]:
import numpy as np
import scipy
from activ import load_data
from activ.readfile import load_preprocessed
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from activ.data_normalization import data_normalization
from scipy.spatial.distance import pdist
from sklearn.model_selection import KFold
import pandas as pd
from activ.readfile import TrackTBIFile

In [116]:
mask = pd.read_csv('/Users/ahyeon/Desktop/TBIanalysis/data/mask_feature_outcome.txt')
mask = mask['0'].values

In [117]:
bm_mask = np.where(mask==0)
oc_mask = np.where(mask==1)

In [118]:
data = load_preprocessed()

In [122]:
full_bm = np.asarray(data.columns[bm_mask])
full_oc = np.asarray(data.columns[oc_mask])

In [123]:
data_bm = np.asarray(data[full_bm])
data_oc = np.asarray(data[full_oc])
type(data_bm)

numpy.ndarray

In [134]:
data_bm

array([[18, 'Male', 'No', ..., 0.0, 0.0, 0.0],
       [49, 'Male', 'No', ..., 0.0, 0.0, 0.0],
       [24, 'Male', 'No', ..., 0.0, 0.0, 0.0],
       ...,
       [55, 'Female', 'No', ..., 0.0, 0.0, 0.0],
       [30, 'Female', 'No', ..., 0.0, 0.0, 0.0],
       [28, 'Male', 'No', ..., 0.0, 0.0, 0.0]], dtype=object)

In [125]:
TrackTBIFile.write('/Users/ahyeon/Desktop/TBIanalysis/data/fulldata.h5', data_bm, data_oc)

TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [58]:
for ii,c in enumerate(data.columns):
    if 'GOSE_Overall' in c:
        print(c,ii)

GOSE_OverallScore3M 352
GOSE_OverallScore6M 405
GOSE_OverallScore12M 615


In [72]:
gose3m = data.iloc[:,352].values.astype(int)
gose6m = data.iloc[:,405].values.astype(int)
gose12m = data.iloc[:,615].values.astype(int)

In [126]:
subdata = load_data()
sub_oc_features = subdata.outcome_features
sub_bm_features = subdata.biomarker_features
sub_oc = subdata.outcomes
sub_bm = subdata.biomarkers

In [132]:
sub_bm[:,2]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
       1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1.,
       1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [80]:
count=0
patients3m = []
patients6m = []

for i in range(586):
    if gose3m[i] == 1 or gose3m[i] == 2:
        patients3m.append(i)
        count+=1
len(patients3m)
# bm_gose3m = np.zeros((len(patients3m), full_bm.shape[1]))
# oc_gose3m = np.zeros((len(patients3m), full_oc.shape[1]))
# for ii,p in enumerate(patients3m):
#     bm_gose3m[ii,:] = full_bm[p,:]
#     oc_gose3m[ii,:] = full_oc[p,:]
# print(bm_gose3m.shape)
# print(oc_gose3m.shape)

27

In [ ]:
fname = '/Users/ahyeon/data/activ/gose1and2/gose1and2.h5'
TrackTBIFile.write(fname, bm_gcs, oc_gcs)

In [ ]:
raw = load_data()
labels = raw.outcome_features
nmfdata = load_data(uoinmf=True)
rawbm = raw.biomarkers #(247,235)
feature_names = raw.outcome_features
A = data_normalization.data_normalization(raw.outcomes, 'positive') #(247,177)
w = nmfdata.outcomes #(247,6)
h = nmfdata.outcome_bases #(6,177)

In [ ]:
def drop_x_percent(A,w,h,feature_names,percent):
    total_features = h.shape[1]
    num_features_to_drop = np.round(total_features*percent//100)
    h_sum = np.sum(h, axis=0)
    h_sum_sorted = np.argsort(h_sum)
    indices_lowest_values = h_sum_sorted[:num_features_to_drop]
    dropped_features = sorted(feature_names[indices_lowest_values])
    h_new = np.delete(h, indices_lowest_values, axis=1)
    A_new = np.delete(A, indices_lowest_values, axis=1) 
    return A_new, h_new, dropped_features

In [ ]:
percent = [0,10,20,30,40,50,60,70,80,90]
for p in percent:
    A_features_dropped,h_new, dropped_features = drop_x_percent(A,w,h,feature_names,p)
    fname = '/Users/ahyeon/data/activ/gcs13-15/bm_gcs_13-15.h5'.format(p)
    TrackTBIFile.write(fname, sub_bm, A_features_dropped)